In [1]:
# load data

!wget https://raw.githubusercontent.com/graykode/commit-autosuggestions/master/experiment/dataset/python/train.jsonl

--2021-11-15 08:54:52--  https://raw.githubusercontent.com/graykode/commit-autosuggestions/master/experiment/dataset/python/train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43686353 (42M) [text/plain]
Saving to: ‘train.jsonl’

train.jsonl         100%[===================>]  41.66M   121MB/s    in 0.3s    

2021-11-15 08:54:53 (121 MB/s) - ‘train.jsonl’ saved [43686353/43686353]

